In [4]:
from util import *
import matplotlib.pyplot as plt

d:\m\File\SKL\A-2024-ZFall\BI CSE 7743 - Machine Learning and Data Science for Bioinformatics\A6\.conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
model_path = "models/run_resnet50_20241208_231720_ep6_best_auc_model.pth"
model = load_model(model_path)
    
# Get balanced samples from train.csv
csv_path = 'deploy/train.csv'
base_path = 'data/png/images_as_pngs_512/train_images_processed_512'
df = get_balanced_samples(csv_path, base_path)

discrimination_line = .65

c:\git\rsna\demo_predictions.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location='cpu'))


In [15]:
df

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case,img_path
40380,1,53636,2044242754,L,MLO,75.0,1,1,0,0.0,0,A,49,False,data/png/images_as_pngs_512/train_images_proce...
21534,2,32911,1468636829,R,MLO,63.0,1,1,1,NaN,0,NaN,48,False,data/png/images_as_pngs_512/train_images_proce...
33265,1,45666,952276083,R,MLO,75.0,1,1,0,0.0,0,B,49,False,data/png/images_as_pngs_512/train_images_proce...
6007,2,16518,914043890,L,CC,51.0,1,1,1,NaN,0,NaN,48,False,data/png/images_as_pngs_512/train_images_proce...
51618,1,6659,734832218,R,MLO,56.0,1,1,1,0.0,0,B,49,False,data/png/images_as_pngs_512/train_images_proce...
14702,2,25679,1160860283,R,CC,52.0,1,1,1,NaN,0,NaN,48,False,data/png/images_as_pngs_512/train_images_proce...
10163,1,20923,1925841866,L,MLO,54.0,1,1,1,0.0,0,B,49,False,data/png/images_as_pngs_512/train_images_proce...
46606,1,6018,640156723,L,MLO,63.0,1,1,1,0.0,0,C,49,False,data/png/images_as_pngs_512/train_images_proce...
25270,1,37032,1243050806,L,MLO,76.0,1,1,1,0.0,0,C,49,False,data/png/images_as_pngs_512/train_images_proce...
35609,2,48171,373646784,L,MLO,72.0,1,1,1,NaN,0,NaN,48,False,data/png/images_as_pngs_512/train_images_proce...


In [22]:
row = df.iloc[10]

image_path = row['img_path']
true_label = row['cancer']

# Preprocess and get prediction
image_tensor, original_image = preprocess_image(image_path)
prob = get_prediction(model, image_tensor)
prob
# discriminate
predicted_label = 'POS' if prob > discrimination_line else 'NEG'
predicted_label


'NEG'

In [24]:
total = len(df)
correct_predictions = 0



for i in range(total):
    row = df.iloc[i]
    image_path = row['img_path']
    true_label = row['cancer']

    # Preprocess and get prediction
    image_tensor, original_image = preprocess_image(image_path)
    prob = get_prediction(model, image_tensor)

    # discriminate
    predicted_label = 1 if prob > discrimination_line else 0
    if predicted_label == true_label:
        correct_predictions += 1

accuracy = correct_predictions / total
accuracy


0.75

In [ ]:

# Add prediction text
annotated_image = add_prediction_text(original_image, prob, true_label)

# Extract filename from path
filename = os.path.basename(image_path)
base_name = os.path.splitext(filename)[0]

# Save annotated image
output_path = f'predictions/{base_name}_pred{prob:.2f}_true{true_label}.png'

# Plot the annotated image
plt.figure(figsize=(8, 8))
plt.imshow(annotated_image)
plt.axis('off')
plt.title(f"Pred: {prob:.2f}, True: {true_label}")
plt.show()
print(f"Processed {output_path}")